# 参考文献


主にhttp://colah.github.io/posts/2015-08-Understanding-LSTMs/
のノーテーションでみていこう。
<img src = 'imgs/LSTM3-chain.png'>



ただし、どの部分がどう呼ばれているかは、
https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

の図表がわかりやすい：

<img src  = 'imgs/1*yBXV9o5q7L_CvY7quJt3WQ.png'>


# LSTM
多分インプット以外は全部同じdimension(?)

- Candidate $C_n$は出力の一つ手前の隠れ層の役割を果たしている
  - $h_n$は出力なのでそのひとつ手前の隠れ層と思えばよい？
  - LSTMの模式図では一番奥に書いてあるが、$x_n \rightarrow C \rightarrow h$と並べたほうが自然（？）
  

- $x_n$,$h_n(=y_n)$に依存する（$C_{n-1}$に依存しない）３つのgateがある (すべてsigmoidで終わる = forgettable)
    - forget gate <img src = 'imgs/LSTM3-focus-f.png'>
    - input gate<img src = 'imgs/LSTM3-focus-i.png'>
    上図の$i_t$を出力とするのがinput gate
    - output gate<img src = 'imgs/LSTM3-focus-o.png'>

- Candidateのバリューはひとつ前$C_{n-1}$から遺伝するが、forget gateを通してボリューム調整する
- 忘却プロセスを経たCandidateをinput gate出力でボリューム調整した「$x_n$,$h_n(=y_n)$に依存する$\tanh$出力」を加算することでアップデートする

<img src = 'imgs/LSTM3-focus-C.png'>

- 上記いずれのプロセスも$C_{n-1}$自体はほとんど用いないことに注意
- $C_n$が決定したら$\tanh$をかけて、output gate出力でボリューム調整して$h_n$($n$ word目の出力）とする

In [27]:
import torch.nn as nn
import torch

In [52]:
input_size = 10
hidden_size = 20
num_layers = 1

In [53]:
rnn = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)

In [30]:
sequence_length = 5
sequence_num = 3

In [31]:
input = torch.randn(sequence_length, sequence_num, input_size)

In [32]:
h0 = torch.randn(num_layers , sequence_num , hidden_size)
c0 = torch.randn(num_layers , sequence_num, hidden_size)
output, (hn, cn) = rnn(input, (h0, c0))

In [54]:
[tuple([x[0] , x[1].shape]) for x in rnn.named_parameters()]

[('weight_ih_l0', torch.Size([80, 10])),
 ('weight_hh_l0', torch.Size([80, 20])),
 ('bias_ih_l0', torch.Size([80])),
 ('bias_hh_l0', torch.Size([80]))]

# GRU

<img src = 'imgs/LSTM3-var-GRU.png'>

- Candidateがない。
- このため、一層の場合、前回の出力$h_{n-1}$と今回の入力$x_n$のみから今回の出力$h_{n-1}$を決めることになる
- これは結構厳しいのではないかという印象を受けるがワークするのはなぜなのだろう？
    - LSTMに劣るという報告もあるようだが（要出典 To be added）

In [ ]:
input_size = 10
hidden_size = 20
num_layers = 1

In [ ]:
rnn = nn.GRU(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)

In [ ]:
sequence_length = 5
sequence_num = 3

In [ ]:
h0 = torch.randn(num_layers , sequence_num , hidden_size)
output, hn = rnn(input, h0)

In [ ]:
1+1


# cell version


sequence長方向のひとつひとつの作用を行う

## GRU
cell state は存在しない

In [ ]:
rnn_cell = nn.GRUCell(input_size = input_size , hidden_size = hidden_size)

In [20]:
input_cell = torch.randn( sequence_num  , input_size)

In [24]:
h0_cell = torch.randn(sequence_num , hidden_size)

In [25]:
rnn_cell(input_cell , h0_cell)

tensor([[-0.8335, -0.2871,  0.6537,  0.4636, -0.2017, -0.4652, -0.0595, -0.6599,
         -0.9843,  0.1885,  0.3383, -0.6164, -1.0814,  0.1980,  0.9775,  0.3202,
          0.5286,  0.5524,  0.2044,  0.1118],
        [ 0.4493, -0.3635,  0.0778, -0.0202,  0.7757,  0.1915,  0.4439,  0.2207,
          1.0404, -0.1683,  0.5032, -0.0317, -0.4123, -1.1217, -0.3295, -0.7662,
         -0.5264, -0.2910,  0.8186,  0.5164],
        [ 0.1806,  0.8379,  1.2413, -0.4384, -0.3630, -0.3390,  0.1768, -0.2117,
         -1.3939, -0.1366,  0.6929, -0.2663, -0.5033,  0.3618,  0.2595, -0.0419,
          0.5714,  0.5075, -0.7026,  0.7079]], grad_fn=<AddBackward0>)

## LSTM

In [26]:
rnn_cell = nn.LSTMCell(input_size=input_size , hidden_size=hidden_size)

In [71]:
c0_cell = torch.randn(sequence_num , hidden_size)

In [81]:
rnn_cell(input_cell , (h0_cell , c0_cell))

NameError: name 'F' is not defined

# customization

https://stackoverflow.com/questions/49040180/change-tanh-activation-in-lstm-to-relu

In [83]:
import torch.nn.functional as F

In [90]:
class my_LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, nlayers, dropout):
        """"Constructor of the class"""
        super(my_LSTMCell, self).__init__()#input_size , hidden_size)
        #super().__init__()#input_size, hidden_size)

        self.nlayers = nlayers
        self.dropout = nn.Dropout(p=dropout)

        ih, hh = [], []
        for i in range(nlayers):
            ih.append(nn.Linear(input_size, 4 * hidden_size))
            hh.append(nn.Linear(hidden_size, 4 * hidden_size))
        self.w_ih = nn.ModuleList(ih)
        self.w_hh = nn.ModuleList(hh)

    def forward(self, input, hidden):
        """"Defines the forward computation of the LSTMCell"""
        hy, cy = [], []
        for i in range(self.nlayers):
            hx, cx = hidden[0][i], hidden[1][i]
            gates = self.w_ih[i](input) + self.w_hh[i](hx)
            i_gate, f_gate, c_gate, o_gate = gates.chunk(4, 1) # chunkは第二引数方向に第一引数分割

            i_gate = torch.sigmoid(i_gate)
            f_gate = torch.sigmoid(f_gate)
            c_gate = F.tanh(c_gate)
            o_gate = torch.sigmoid(o_gate)

            ncx = (f_gate * cx) + (i_gate * c_gate)
            nhx = o_gate * F.tanh(ncx)
            cy.append(ncx)
            hy.append(nhx)
            input = self.dropout(nhx)

        hy, cy = torch.stack(hy, 0), torch.stack(cy, 0)
        return hy, cy

chunkは第二引数方向に第一引数分割

In [91]:
rnn_cell = my_LSTMCell(input_size=input_size , hidden_size=hidden_size , nlayers=1 , dropout = 0)

In [92]:
rnn_cell(input_cell , (h0_cell , c0_cell))

(tensor([[[ 0.0153, -0.0195,  0.0217, -0.1277, -0.1335,  0.6000,  0.0666,
           -0.1095, -0.0361,  0.2820, -0.3620,  0.4399, -0.2040,  0.1220,
           -0.0065, -0.0982, -0.3041,  0.2062,  0.1905, -0.5630],
          [ 0.1604, -0.2095,  0.2442,  0.0333,  0.0181,  0.2809, -0.0675,
           -0.1081,  0.0338,  0.2612, -0.3391,  0.4058, -0.1925,  0.0652,
            0.0232,  0.1189, -0.0761, -0.6512,  0.1925, -0.3242],
          [-0.0127, -0.1873,  0.0176, -0.0264, -0.0349,  0.3741,  0.0855,
           -0.2058, -0.0437,  0.4875, -0.3991,  0.4650, -0.4058,  0.1545,
            0.0276,  0.0218,  0.0465,  0.0267,  0.4557, -0.4281]]],
        grad_fn=<StackBackward>),
 tensor([[[ 0.0425, -0.1363,  0.1183, -0.8400, -0.3843,  0.9999,  0.0893,
           -0.9688, -0.0657,  0.8581, -0.5932,  0.7532, -0.3199,  0.1576,
           -0.0134, -0.3333, -0.4158,  0.3860,  0.6042, -0.9542],
          [ 0.2280, -0.3540,  0.5198,  0.0738,  0.1099,  0.7289, -0.1476,
           -0.4229,  0.0836,  0.34